# Improving public policy by predicting car accident severity using machine learning

This notebook will be used to develop the capstone project of the IBM Certification in Data Science.

The purpose of this project is to predict accident using attributes collected by the Seattle Polive Department and recorded by Traffic Records.


Original dataset: [link_dataset](https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv)  
Metadata: [link_metadata](https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Metadata.pdf)

In order to get the most out of this dataset, I followed the 


---
## Table of Contents

### 1 - [Introduction/Business Problem](#business-und)

### 2 - [Data understanding and Data Preparation](#data-und-data-prep)

### 3 - [Modeling](#modeling)

### 4 - [Evaluation](#evaluation)

### 5 - [Conclusion](#conclusion)
---

In [ ]:
##### Import necessary packages
import sklearn
import numpy as np
import scipy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-dark')

# Ploting params
plt.rcParams['axes.titlesize'] = 14
# axes.titlesize : 24
# axes.labelsize : 20
# lines.linewidth : 3
# lines.markersize : 10
# xtick.labelsize : 16
# ytick.labelsize : 16

<a id="business-und"><a/>

# 1 - Introduction/Business Problem
---- 


**Disclaimer:** _This project is part of the Coursera Capstone to complete the Data Science Certification. All project ideas are entirely fictitious and serve only the purpose of developing a data science project._

#### Predicting accident severity in US cities based on specific metrics can be a powerful tool to drive public policy and reduce overall accident rate. In Seattle – Washington, the Department of Transport/Traffic Management Division,  has been collecting data since 2004 about collisions in the metropolitan area with the objective of creating a complete database that represents the overall road accidents involving collisions in this city. The City Council of Seattle has the responsibility of approving the city's budget, and develops laws and policies intended to promote safety of Seattle's residents  . During every fiscal year, the city council discusses policies to improve road safety in Seattle by reducing the number of human injuries involved in those accidents. 
#### In this data science project, I proposed to develop a model that can distinguish accidents resulting in human injuries from accidents resulting in property damage-only. This prediction will be based on widely available metrics provided by the Department of transport and it can help to identify which factors may increase the risk for injury-related accidents and help develop actions to reduce those. The successful outcome of this project would be a model that can predict with accuracy accidents associated with human costs (i.e. high true positive rate), so that actions can be developed to minimize those costs. Examples of actions could include target advertising for road safety, improve road design, increase police deployment to secure roads, increase fines for reckless driving, among others. 


<a id="data-und-data-prep"><a/>

# 2 - Data understanding and Data Preparation
---
    
#### To develop my data project, I will use data about collisions provided by Seattle Police Department and recorded by Traffic Records . This dataset includes all types of collisions happening at intersection or mid-block of a road segment since 2004 and contains information about many important factors such as road condition and lightning conditions, weather, segment of the road involved (among other) and associated with each accident there is a variable that represents the outcome severity with 2 values: type 1 – property damage-only and type 2 – Injury-related. By using this information, I will develop a classification model aiming at predicting the severity outcome of the accident, with particular emphasis at predicting type 2 accidents.  I will focus on optimizing the model to get a high true positive rate of detection of type 2 accidents and the results obtained from this model could guide actions to decrease the occurrence of these accidents. 

In [ ]:
# specify the location of the data
data_url = 'https://github.com/joseferncruz/coursera-capstone/raw/master/datasets/collisions_data.csv'

# load the data into a dataframe
df_raw = pd.read_csv(
    data_url,
    parse_dates=['INCDTTM', 'INCDATE'], # Parse dates to datetime objects 
    usecols=[
        'SEVERITYCODE', 'X', 'Y', 'ADDRTYPE', 'PERSONCOUNT', 'PEDCOUNT',
        'PEDCYLCOUNT', 'VEHCOUNT', 'INCDTTM', 'JUNCTIONTYPE', 'WEATHER',
        'ROADCOND', 'LIGHTCOND', 'CROSSWALKKEY', 'INCDATE',
    ],
    low_memory=False,
);

#### Let's start by exploring the features and characteristics of this dataset.

In [ ]:
print(f'This dataset has {df_raw.shape[0]} rows and {df_raw.shape[1]} columns, including one target column.')

In [ ]:
# display some information about the datatypes and number of entries associated with the full raw dataset
df_raw.info()

By exploring the metadata, it is possible to notice that some columns have large quantities of missing data or lack information (Not Enough Information or NEI). Also our model should be able to predict accident severity based on features (ie information) than can be measured in real time or within hour range (eg weather, road conditiong, etc).  

In [ ]:
# Drop columns not used in this case study
df = df_raw.copy()

#### In order to solve this classification problem, I will use the following features from the original dataset:


|Attributes| Description|  
|---:|:----|  
|SEVERITYCODE|A code that corresponds to the severity of the collision|  
| X | GPS Latitude coordinate |
| Y | GPS Longitude coordinate | 
|ADDRTYPE|Collision address type|
|PERSONCOUNT|The total number of people involved in the collision|
|PEDCOUNT|The number of pedestrians involved in the collision |
|PEDCYLCOUNT| The number of bicycles involved in the collision. |
|VEHCOUNT|The number of vehicles involved inthe collision|
|INCDTTM|The date and time of the incident.|
| JUNCTIONTYPE| Category of junction at which collision took place |
|WEATHER|A description of the weather conditions during the time of the collision|
|ROADCOND|The condition of the road during the collision|
|LIGHTCOND|The light conditions during the collision|
|CROSSWALKKEY| A key for the crosswalk at which the collision occurred |


In [ ]:
df.columns

#### The first step is to remove missing/unknown information from columns in order to reduce the uncertainty around certain features. In this project I will consider only observations that have complete, unambigous information. 

In [ ]:
# Remove entries with missing information
df.dropna(inplace=True)

# Remove missing/uncertain values from WEATHER
cond = df['WEATHER'].isin(['Unknown', 'Other'])
df = df.loc[~cond, :].copy()

# Remove missing/uncertain values from ROADCOND
cond = df['ROADCOND'].isin(['Unknown', 'Other'])
df = df.loc[~cond, :].copy()

# Remove missing/uncertain values from LIGHTCOND
cond = df['LIGHTCOND'].isin(['Unknown', 'Other', 'Dark - Unknown Lighting'])
df = df.loc[~cond, :].copy()

# JUNTIONTYPE
cond = df['JUNCTIONTYPE'].isin(['Unknown'])
df = df.loc[~cond, :]

# Remove alleys from address type due to low samples
df = df.loc[~df['ADDRTYPE'].isin(['Alley']), :].copy()

# remove events with missing hour 
# Extract the hour
df['HOUR'] = df['INCDTTM'].dt.hour
# remove faulty entries (ie not correct hour record)
cond = df['HOUR'].isin([0])
df = df.loc[~cond, :].copy()

In [ ]:
# Display information about the dataset without missing data or incomplete information.
df.info()

The `SEVERITYCODE` target variable can take 2 values (1/2):
- 1: **Property Damage**  
- 2: **Injury**




In [ ]:
# How many observations per target category exists?
for category, counts in df.SEVERITYCODE.value_counts().items():
    print(f"There are {counts} elements in target value {category}.")
    
# Print normalized count in each category of severity.
norm_count = df.SEVERITYCODE.value_counts(normalize=True).to_dict()
print(f"The dataset is devided into{norm_count.get(1)*100: 0.2f}% Severity 1 and{norm_count.get(2)*100: 0.2f}% Severity 2 observations.")

#### As it is possible to observe, our dataset is quite imbalanced and this poses a challenge to classification algorithms. For now, we will consider the full dataset (ie records without missing or ambigous information) and I will address later the problem of the imbalanced data.

# Location of type 1 and type 2 severity events in the city of Seattle

#### Since we have GPS Data, the first step will be to cluster the city of seattle into 10 different spatial clusters and identify those clusters. This information can also be relevant for the sponsors to target their actions to reduce accident severity.

In [ ]:
df.columns

## Divide the city of Seattle into 12 clusters

In [ ]:
from sklearn.cluster import KMeans

# Initiate KMeans for 10 clusters (ie putative city areas)
kmeans = KMeans(n_clusters = 12, init ='k-means++', random_state=32)

# Get coords dataframe
coords = df[['X', 'Y']]

# Fit the model
kmeans.fit(coords) # Compute k-means clustering.

# get the labels
labels = kmeans.predict(coords) # Labels of each point

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20, 10), sharey=True)

# Set condition to access both types of severity independently
cond = df['SEVERITYCODE'] == 1

# Class 1
ax[0].plot(df['X'][cond], df['Y'][cond], '.', ms=2, alpha=0.5, color='black')
ax[0].set(ylabel='Latitude')
ax[0].legend(labels=['Severity 1'])

# Class 2
ax[1].plot(df['X'][~cond], df['Y'][~cond], '.', ms=2, color='black', alpha=0.5)
ax[1].legend(labels=['Severity 2'])

# Both 1 and 2
for event_type in [1, 2]:
    ax[2].plot(df['X'][df['SEVERITYCODE'] == event_type], df['Y'][df['SEVERITYCODE'] == event_type], '.', ms=3, alpha=0.5)
ax[2].legend(labels=['Severity 1', 'Severity 2'], loc='upper right')

# Complete plot info
for idx in range(3):
    ax[idx].set(title='Seattle')
    ax[idx].set(xlabel='Longitude')
    ax[idx].tick_params('x', labelrotation=45)
    
    coords.plot.scatter(x = 'X', y = 'Y', c=labels, s=50, cmap='viridis', ax=ax[idx], alpha=0.5)

In [ ]:
# Create a new feature: `SECTOR_ID` with labels from the clustering algorithm.
df['SECTOR_ID'] = labels

In [ ]:
# Check the total numbe of severity 1 and 2 events for each sector
result = df.groupby(['SEVERITYCODE'])['SECTOR_ID'].value_counts(normalize=True).to_frame('NORM_COUNT').reset_index()
result.sort_values('NORM_COUNT', inplace=True)

In [ ]:
sns.barplot(x='SECTOR_ID', y='NORM_COUNT', data=result, hue='SEVERITYCODE', )

#### It seems that Sector 5 and 7 have higher proportion of accident severity 2. This information can be used with one-hot encoder to create 2 more variables `SECTOR_5` and `SECTOR_7`

In [ ]:
df['SECTOR_5'] = df['SECTOR_ID'].apply(lambda x: 1 if x == 5 else 0)
df['SECTOR_7'] = df['SECTOR_ID'].apply(lambda x: 1 if x == 5 else 0)

## When do events of specific severities happen more often?
---
#### In order to answer this question, we need to create new time features from the time feature in `INCDTTM`.

In [ ]:
# Extract the week day
df['WEEKDAY'] = df['INCDTTM'].dt.weekday

# Extract the hour
df['HOUR'] = df['INCDTTM'].dt.hour

# Extrack the month
df['MONTH'] = df['INCDTTM'].dt.month

# Extract the year
df['YEAR'] = df['INCDTTM'].dt.year

# Display top 3 columns
df.head(3)

In [ ]:
# Extract monthly normalized count of events for each category
result = df.groupby('SEVERITYCODE')['MONTH'].value_counts(normalize=True).to_frame('NORM_COUNT').reset_index()


fig, ax = plt.subplots()
sns.barplot(x='MONTH', y='NORM_COUNT', data=result, hue='SEVERITYCODE', ax=ax)

month_labels = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
ax.set(ylim=(0, 0.15), xticklabels=month_labels, title='Monthly Accident proportion in each Severity Category');

#### It seems that **JULY** and **AUGUST** have a (low, but still) slightly higher proportion of accidents in the category 2. We could used this information to create one additional one-hot encoded variable `SUMMER`.

In [ ]:
df['SUMMER'] = df['MONTH'].apply(lambda x: 1 if x in [6, 7] else 0)

In [ ]:
# Show the proportion.
df.groupby('SEVERITYCODE')['SUMMER'].value_counts(normalize=True)

## What about daily modulation in accident severity? Is there a particular time of the day with higher proportion of accidents belonging to a specific severity?
----


In [ ]:
# Estract normalized count of each severity category for each hour
result = df.groupby('SEVERITYCODE')['HOUR'].value_counts(normalize=True).to_frame('NORM_COUNT').reset_index()

fig, ax = plt.subplots()

sns.barplot(x='HOUR', y='NORM_COUNT', data=result, hue='SEVERITYCODE', ax=ax)

ax.set(title='Hourly accident proportion in each severity Category')

#### It seems that during the afternoon there higher a higher proportion of accidents belonging to category 2, particularly during the rush rour (5-7PM). Let's capture this variation in another one-hot encoded variable `AFTERNOON`. 

In [ ]:
df['AFTERNOON'] = df['HOUR'].apply(lambda x: 1 if x in [14, 15, 16, 17, 18] else 0)

df.groupby('SEVERITYCODE')['AFTERNOON'].value_counts(normalize=True)

## What is the average human cost associated with each city sector?
---
#### In order to maximize the clustering information, it would be good to find a measure of human cost that could be used to describe each individual group (ie `SECTOR_ID`). One way to do this is to explore a measurement of the ratio between of humans vs vehicles involved. 

In [ ]:
# Create a single column that agregates all the non-motorized human involvement
df['TOTAL_HUMAN'] = df[['PERSONCOUNT', 'PEDCYLCOUNT', 'PEDCOUNT']].sum(axis=1)
df['TOTAL_HUMAN'].head()

## What is the relationship between total number of people involved and total number of vehicles involved?  

#### Before dwelling into to the relation, let's have a look at the distribution of values within each variable.

In [ ]:
# Some stats about severity and location
df.groupby(['SEVERITYCODE'])[['VEHCOUNT', 'TOTAL_HUMAN']].agg([np.mean, np.min, np.max])

#### It seems that a higher number of people is associated with a severity of type 2, alghouth the mean is only slightly lower. Let's look at the distribution.

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20, 5))
fig.suptitle('Distribution of People involved in Accidents.')

bins = 10 

ax[0].hist(df['TOTAL_HUMAN'][df['SEVERITYCODE']==1], log=True, bins=bins, color='#1f77b4', label='Severity 1', normed=True);
ax[1].hist(df['TOTAL_HUMAN'][df['SEVERITYCODE']==2], log=True, bins=bins, color='#ff7f0e', label='Severity 2', normed=True);

ax[2].hist(df['TOTAL_HUMAN'][df['SEVERITYCODE']==1], log=True, bins=bins, color='#1f77b4', label='Severity 1', normed=True);
ax[2].hist(df['TOTAL_HUMAN'][df['SEVERITYCODE']==2], log=True, bins=bins, color='#ff7f0e', label='Severity 2', normed=True);

for idx in range(3):
    ax[idx].legend()
    ax[idx].set(ylabel='count (log)', xlabel='People')

In [ ]:
result = df.groupby(['SEVERITYCODE', 'SECTOR_ID']).mean()[['TOTAL_HUMAN', 'VEHCOUNT']].reset_index()

In [ ]:
# Plot the relation between total number of people involved vs vehicles.
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
fig.suptitle('Relation between Vehicle Number and Humans involved.')

sns.regplot(
    x='TOTAL_HUMAN',
    y='VEHCOUNT',
    data=result,
    order=1,
    ax=ax[0]
)
ax[0].set(
    ylabel='Mean Vehicle Number',
    xlabel='Mean Human Number',
    title='Linear Regression between vehicles and humans')

sns.residplot(
    x='TOTAL_HUMAN',
    y='VEHCOUNT',
    data=result,
    order=1,
    ax=ax[1]
)
ax[1].set(
    title='Residual plot',
    ylabel='Residuals from the model',
    xlabel='',
    ylim=(-0.15, 0.15));

### Are the two variables correlated?
----

In [ ]:
slope, intercept, rvalue, pvalue, stderr = scipy.stats.linregress(result[['TOTAL_HUMAN', 'VEHCOUNT']])

print(f"The correlation coeff (person) is: {rvalue: 0.2f} and the p-value associated is {pvalue: 0.3f}")

#### The two variables are negatively correlated, meaning that there is a higher number of people associated with accidents involving fewer vehicles. 

## What is the ratio between total number of people involved and number of vehicles?
---

#### Let's now calculate the ratio between people involved and vehicles involved for each observation in the dataset.



In [ ]:
# Add small value to VEH count to avoid ZeroDivisionError 
df[['VEHCOUNT']] = df[['VEHCOUNT']] + 0.001

# Divide total_human by veh_count
df['RATIO_HUMAN/VEH'] = df['TOTAL_HUMAN'].div(df['VEHCOUNT'])

#### Now let's check the mean of the `RATIO_HUMAN/VEH` of each Cluster and for each severity catergory.

In [ ]:
human_cost = df.groupby(['SEVERITYCODE', 'SECTOR_ID']).mean()['RATIO_HUMAN/VEH'].to_frame().reset_index()

human_cost.head()

#### Let's Create a new feature called `SECTOR_HUMAN_COST` which stores the score of human cost for each sector.

In [ ]:
severity_code_1 = human_cost[human_cost['SEVERITYCODE']==1][['SECTOR_ID', 'RATIO_HUMAN/VEH']]
severity_code_1 = severity_code_1.to_dict().get('RATIO_HUMAN/VEH')

severity_code_2 = human_cost[human_cost['SEVERITYCODE']==2][['SECTOR_ID', 'RATIO_HUMAN/VEH']].reset_index()
severity_code_2 = severity_code_2.to_dict().get('RATIO_HUMAN/VEH')

In [ ]:
# 
df['SECTOR_HUMAN_COST_a'] = df[df['SEVERITYCODE']==1]['SECTOR_ID'].replace(severity_code_1)
df['SECTOR_HUMAN_COST_a'].fillna(0, inplace=True)

In [ ]:
df['SECTOR_HUMAN_COST_b'] = df[df['SEVERITYCODE']==2]['SECTOR_ID'].replace(severity_code_2)
df['SECTOR_HUMAN_COST_b'].fillna(0, inplace=True)

In [ ]:
df['SECTOR_HUMAN_COST'] = df['SECTOR_HUMAN_COST_a'] + df['SECTOR_HUMAN_COST_b']

In [ ]:
sns.barplot(x='SEVERITYCODE', y='SECTOR_HUMAN_COST', data=df, hue='SECTOR_ID')

#### Create a unifrom value to define each cluster.

In [ ]:
net_human_cost = human_cost['RATIO_HUMAN/VEH'][human_cost['SEVERITYCODE']==2].reset_index(drop=True)\
                 - human_cost['RATIO_HUMAN/VEH'][human_cost['SEVERITYCODE']==1]

In [ ]:
# Set up new variable 
df['SECTOR_HUMAN_COST'] = df['SECTOR_ID'].replace(net_human_cost.to_dict())

In [ ]:
# Plot the results 
sns.barplot(
    x='SECTOR_ID',
    y='SECTOR_HUMAN_COST',
    data=df
)

In [ ]:
df['TOTAL_INTERV'] = df['TOTAL_HUMAN'] + df['VEHCOUNT']

## Are specific types of road structures involved in specific types of events?
---
#### Among the data choosen to build the model, we can find information about types of collision addresses: Intersection and Block. 


In [ ]:
result = df.groupby(['SEVERITYCODE'])['ADDRTYPE'].value_counts(normalize=True).to_frame('NORM_COUNT').reset_index()

fig, ax = plt.subplots()
sns.barplot(
    data=result, 
    x='ADDRTYPE', 
    y='NORM_COUNT', 
    hue='SEVERITYCODE',
    ax=ax
)

ax.set(
    title='Accident severity and collision address type', 
    ylabel='Normalized count', 
    xlabel='Collision address type',
    ylim=(0, 1)
)

#### It seems that Severity type 2 accidents happen in equally proportions in both Blocks and Intersections whereas type 1 accidents tend to happen more in Blocks.

## What about the road condition? How does road condition relate to accident severity?
---
There are several different attributes in terms of road condition, for instance:

In [ ]:
print('Different states of road conditions', df.ROADCOND.unique())

In [ ]:
result = df.groupby(['SEVERITYCODE'])['ROADCOND'].value_counts(normalize=True).to_frame('NORM_COUNT').reset_index()


fig, ax = plt.subplots()

sns.barplot(
    data=result, 
    x='ROADCOND', 
    y='NORM_COUNT', 
    hue='SEVERITYCODE',
)
ax.tick_params('x', labelrotation=45)
ax.set(title='Road conditions and accident severity', ylabel='normalized count', xlabel='')

#### It seems that there are no striking differences between road conditions and the different types of accident severity. In order to decrease the number of different categories, we will bin some of the variables with similar conditions.

In [ ]:
dry = ['Dry']
wet = ['Wet', 'Ice', "Snow/Slush", 'Sand/Mud/Dirt', 'Standing Water', 'Oil']


for word in wet:
    df['ROADCOND'] = df['ROADCOND'].replace(word, 'wet')
for word in dry:
    df['ROADCOND'] = df['ROADCOND'].replace(word, 'dry')
    
df.groupby('SEVERITYCODE').ROADCOND.value_counts(normalize=True)

In [ ]:
df.groupby(['SEVERITYCODE'])['LIGHTCOND'].value_counts(normalize=True)

#### Another important component that can play a role in accident severity is the lighting conditiong at the time of the accident. 
## Is there any relationship between accident severity and luminosity at the time of the accident?
---
Similarly to road condition, there are several different values associated with this feature. 

In [ ]:
df.groupby('SEVERITYCODE').LIGHTCOND.value_counts(normalize=True)

#### Let's bin the values by good and bad visibility condiitons.

In [ ]:
# Group by similar condtions
day_light = ['Daylight']
low_light = ['Dark - Street Lights On', 'Dusk', 'Dawn', 'Dark - No Street Lights', 'Dark - Street Lights Off']

for word in day_light:
    df['LIGHTCOND'] = df['LIGHTCOND'].replace(word, 'light-good')
for word in low_light:
    df['LIGHTCOND'] = df['LIGHTCOND'].replace(word, 'light-bad')
    
df.LIGHTCOND.unique()

In [ ]:
result = df.groupby(['SEVERITYCODE'])['LIGHTCOND'].value_counts(normalize=True).to_frame('NORM_COUNT').reset_index()


fig, ax = plt.subplots()

sns.barplot(
    data=result, 
    x='LIGHTCOND', 
    y='NORM_COUNT', 
    hue='SEVERITYCODE',
)

ax.set(ylim=(0, 1), title='Light conditions and accident severity', ylabel='normalized count', xlabel='')

#### It seems that good lightning conditions have slightly higher proportion of accidents of type 2 (ie injury-related). It could be explained for instance buy driver over confidence during a seemingly safer road period (ie daylight).

---
#### While we have observed that Insersections have higher proportion of type 1 accidents, We can explore the feature `JUNCTIONTYPE` to extract more insights about the properties of the accidents.

In [ ]:
# Junction Type
df.groupby('SEVERITYCODE').JUNCTIONTYPE.value_counts(normalize=True)

#### It seems that type 1 Severity Accidentds happen 50% of the times at Mid-Blocks whereas type 2 accidents happen with ~48% of the times at Intersection (intersection related). Since getting higher positive rate of type 2 accidents is our priority, I will use this information to create a one-hot encoded feature `INTERSECTION_RELATED` with this informatinon: 

In [ ]:
df['INTERSECTION_RELATED'] = df['JUNCTIONTYPE'].apply(lambda x: 1 if x == 'At Intersection (intersection related)' else 0)

## How is weather forecast related to accident severity type?
---
The weather is also an important aspect of safe driving conditions and can have a strong impact on the type of accidents that can place. In our dataset, weather can have the following values:

In [ ]:
df['WEATHER'].unique()

In [ ]:
result = df.groupby(['SEVERITYCODE'])['WEATHER'].value_counts(normalize=True).to_frame('NORM_COUNT').reset_index()


fig, ax = plt.subplots()

sns.barplot(
    data=result, 
    x='WEATHER', 
    y='NORM_COUNT', 
    hue='SEVERITYCODE',
)

ax.set(ylim=(0, 0.7), title='Weather and accident severity', ylabel='normalized count', xlabel='');
ax.tick_params('x', labelrotation=30)

In [ ]:
df.groupby('SEVERITYCODE').WEATHER.value_counts(normalize=True)

#### The vast majority of the accidents (>65%) happen during periods of Clear sky. Let's bin the different categories in: `sunny`, `cloudy` and `rainy-snow`.

In [ ]:
# Eeather categories
sunny = ['Clear', ]
cloudy = ['Overcast', 'Fog/Smog/Smoke', 'Partly Cloudy', 'Blowing Sand/Dirt', 'Severe Crosswind', ]
rainy_snow = ['Raining', 'Snowing', 'Sleet/Hail/Freezing Rain', ]

for word in sunny:
    df['WEATHER'] = df['WEATHER'].replace(word, 'sunny')
for word in cloudy:
    df['WEATHER'] = df['WEATHER'].replace(word, 'cloudy')
for word in rainy_snow:
    df['WEATHER'] = df['WEATHER'].replace(word, 'wet')

In [ ]:
result = df.groupby(['SEVERITYCODE'])['WEATHER'].value_counts(normalize=True).to_frame('NORM_COUNT').reset_index()


fig, ax = plt.subplots()

sns.barplot(
    data=result, 
    x='WEATHER', 
    y='NORM_COUNT', 
    hue='SEVERITYCODE',
)

ax.set(ylim=(0, 0.7), title='Weather and accident severity', ylabel='normalized count', xlabel='weather forecast');

## Accidents at Crosswalks
---
#### Finaly and to complete this basic EDA, let's have a look at the `CROSSWALKKEY` feature. This feature hold the key of the crossroad that is associated with an accident. Let's investigate whether accidents at crosswalks are associated with specific types of accidents. 

In [ ]:
# Transform the variable in 1 where there is an event at crossroad (ie accident) or 0 elsewhere.
df['CROSSWALKKEY'] = df['CROSSWALKKEY'].apply(lambda x: 0 if x == 0 else 1)

df.groupby('SEVERITYCODE')['CROSSWALKKEY'].value_counts(normalize=True)

#### It seems that the majority of the accidents do not take place at crossroads. However, severity 2 accidents tend to associated in higher proportion to crossroads compared to severity 1 accidents.

Select and prepare data for modeling.
---

In [ ]:
df = df[['SEVERITYCODE', 'ADDRTYPE', 'WEATHER', 'ROADCOND', 'LIGHTCOND', 'CROSSWALKKEY',
         'SECTOR_5', 'SECTOR_7', 'WEEKDAY', 'MONTH', 'INTERSECTION_RELATED',
         'AFTERNOON', 'SUMMER', 'TOTAL_HUMAN', 'SECTOR_HUMAN_COST']]

df.info()

### One-hot encoding of categorical data
---

In [ ]:
df = pd.get_dummies(df)
df.info()

### Balance the dataset
---
#### Implementing a resampling method of the majority class.

In [ ]:
from imblearn.under_sampling import RandomUnderSampler # https://imbalanced-learn.readthedocs.io/en/stable/api.html

# Create resampling object 
rus = RandomUnderSampler(replacement=False, random_state=32, sampling_strategy='majority')

# create target bool array to subset dataset
target_cond = df.columns == 'SEVERITYCODE'

# Undersample the data
FEATURES, TARGET = rus.fit_sample(df.loc[:, ~target_cond], df.loc[:, target_cond])

# Reassemble the dataset for preprocessing
FEATURES['SEVERITYCODE'] = TARGET

# Reassign variable to df
df = FEATURES.copy()

# Show the information about undersampled dataset
df.info()

<a id="modeling"><a/>

# Modeling
---

In this problem, I intend to build a binary classifier model based on previous observations. Thus, this problems represents a supervised learning problem and in order to tackle my classification problem I will use 3 types of machine learning algorithms:

- Logistic Regression
- Gradient Boosting Classifier
- Random Forest ensemble

[Descriptin about the method and implementatio]
    

In [ ]:
# Import models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Import preprocessing tools and model selection
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, KFold, cross_val_score, KFold

## Split data into Train and Test datasets

In [ ]:
# Features used to generate the model
FEATURES = df.loc[:, df.columns != 'SEVERITYCODE']

# What is intended to be predicted
TARGET = df.loc[:, ~(df.columns != 'SEVERITYCODE')].replace({1: 0, 2: 1}).to_numpy().flatten()

# Split the data into train/test data
X_train, X_test, y_train, y_test = train_test_split(FEATURES, TARGET, random_state=32, test_size=0.15)

## Scale the data
#### This step is important for the Logistic Regression Model although it would be unecessary for Decision Tree based models. 

In [ ]:
# Use a standard scaler on the data
scaler = StandardScaler()

# scale train data
X_train = scaler.fit_transform(X_train)

# Scale test data with scaler object fit with train data
X_test = scaler.transform(X_test)

## Create dummy classifier
#### The purpose of the dummy classifier is to assess how much of my model predicts are attributed to chance. For that I will implement and use a dummy classifier.

In [ ]:
from sklearn.dummy import DummyClassifier

# Create the dummy classifier
dummy_clf = DummyClassifier(strategy='uniform', random_state=32)

# Fit Data
dummy_clf.fit(X_train, y_train)

# Predict
y_dummy = dummy_clf.predict(X_test)

## Generalized Linear Model for Classification: Logistic Regression.




#### Let's use grid search cross validation to find the best params

In [ ]:
# Initiate the Logistic regression model
logreg = LogisticRegression(random_state=32, max_iter=200)

# Implement stratified Cross Validation
Kfold = KFold(n_splits=5)

# Find the best parameters using gridsearch cross validation
params_grid = [{ 
    'solver': ['liblinear'],'penalty': ['l2'], 'C': [0.01, 0.1, 1, 10, 100]
},
    {
        'solver': ['saga'], 'penalty': ['l1', 'l2'], 'C': [0.01, 0.1, 1, 10, 100]
    }
]

# Instantiate grid search cross validation object
gs_logreg = GridSearchCV(logreg, params_grid, cv=Kfold);

# Fit the model
gs_logreg.fit(X_train, y_train);

In [ ]:
print(f'The best score for the GridSearch CrossValidation was{gs_logreg.best_score_: 0.2f}')

In [ ]:
# print the best params
gs_logreg_best_params = gs_logreg.best_params_

print(gs_logreg_best_params)

In [ ]:
# Initiate the Logistic regression model
logreg = LogisticRegression(
    penalty=gs_logreg_best_params.get('penalty'),
    C=gs_logreg_best_params.get('C'),
    solver=gs_logreg_best_params.get('solver'),
)

# Fit the training data
logreg.fit(X_train, y_train)

# Predict using logreg
y_logreg = logreg.predict(X_test)

## Random Forest Classifier

In [ ]:
# Initiate the Logistic regression model
forest = RandomForestClassifier(random_state=32, n_jobs=-1)

# Implement stratified Cross Validation
kfold = KFold(n_splits=5)

# Find the best parameters using gridsearch cross validation
params_grid = { 
    'n_estimators': [300],
    'max_depth': [4, 5, 6],
#     'min_samples_leaf': [1, 2, 4],
#     'min_samples_split': [2, 5, 10],
}

# Instantiate grid search cross validation object
gs_forest = GridSearchCV(forest, params_grid, cv=Kfold);

# Fit the model
gs_forest.fit(X_train, y_train);

In [ ]:
gs_forest.best_params_

In [ ]:
# get the best parameters from the grid search
forest_best_params = gs_forest.best_params_

# Instantiate a new model
forest = RandomForestClassifier(
    n_estimators=forest_best_params.get('n_estimators'), 
    max_depth=forest_best_params.get('max_depth'), 
    random_state=32,
    n_jobs=-1,
)

# Fit the data 
forest.fit(X_train, y_train)

# Predict
y_forest = forest.predict(X_test)

### Gradient Boosting Classifier
small intro

In [ ]:
# # Initiate the Logistic regression model
# gbrt = GradientBoostingClassifier(random_state=32)

# # Implement stratified Cross Validation
# kfold = KFold(n_splits=5)

# # Find the best parameters using gridsearch cross validation
# params_grid = { 
#     'n_estimators': [10, 100], 'max_depth': [4, 8], 'learning_rate': [0.1, 0.01, 1]
# }

# # Instantiate grid search cross validation object
# gs_gbrt = GridSearchCV(gbrt, params_grid, cv=Kfold);

# # Fit the model
# gs_gbrt.fit(X_train, y_train);

In [ ]:
# gs_gbrt.best_params_

In [ ]:
# Get the best paramters
#gbrt_best_params = gs_gbrt.best_params_

# Instantiate a new object
gbrt = GradientBoostingClassifier(learning_rate=0.1, max_depth=4)#max_depth=gbrt_best_params.get('max_depth'), n_estimators=gbrt_best_params.get('n_estimators'), learning_rate=gbrt_best_params.get('learning_rate'))

# fit data
gbrt.fit(X_train, y_train)

# predict
y_gbrt = gbrt.predict(X_test)

<a id="evaluation"><a/>
# Evaluation
    
There are several ways to ways to score classification problems such as:
- simple accuracy
- jaccard index
- f1-scores
    

For our problem, identifying true positives (true category one accident or true cattegory 2 accident) is more important than true negatives since [COMPLETE]
I will also compare my results to chance by using the dummy classifier.
For this binary classification problem, I will use precision and recall measurements, associated F1 scores and Receiver Operand Curves (ROC).

In [ ]:
# Import metrics
from sklearn.metrics import jaccard_score, f1_score, confusion_matrix, classification_report,\
                            roc_curve, plot_confusion_matrix, plot_roc_curve, plot_precision_recall_curve

#### Let's look at the different f1 scores for each estimator

In [ ]:
print(f'   Logistic Regression: F1 Score on test set: {f1_score(y_test, y_logreg,): 0.2f}')
print(f'Random Forest Ensemble: F1 Score on test set: {f1_score(y_test, y_forest,): 0.2f}')
print(f'     Gradient Boosting: F1 Score on test set: {f1_score(y_test, y_gbrt,): 0.2f}')
print(f'         Decision Tree: F1 Score on test set: {f1_score(y_test, y_dummy,): 0.2f}')

#### To better undertand the F1 scory_gbrt we can have a look at the `classification_report` and corresponding `confusion_matrix`

In [ ]:
print("Logistic Regresion\n--------------------\n", classification_report(y_test, y_logreg))
print("Random Forest\n--------------------\n", classification_report(y_test, y_forest))
print("Gradient Boosting\n---------------------\n", classification_report(y_test, y_gbrt))
print("Decision Tree\n---------------------\n", classification_report(y_test, y_dummy))

In [ ]:
print("Logistic Regresion\n--------------------\n", classification_report(y_test, y_logreg))
print("Random Forest\n--------------------\n", classification_report(y_test, y_forest))
print("Gradient Boosting\n---------------------\n", classification_report(y_test, y_gbrt))

### `plot_confusion_matrix`

In [ ]:
# Confusion Matrix
fig, ax = plt.subplots(1, 3, figsize=(20, 5))

for idx, (name, estimator) in enumerate({'Logistic Regression': logreg, 'Random Forest': forest, 'Gradient Boosting':gbrt}.items()):
    plot_confusion_matrix(estimator, X_test, y_test, ax=ax[idx])
    ax[idx].set(title=f'{name}')

### `plot_precision_recall_curve`

In [ ]:
# Confusion Matrix
fig, ax = plt.subplots(1, 3, figsize=(20, 5))
for idx, (name, estimator) in enumerate({'Logistic Regression': logreg, 'Random Forest': forest, 'Gradient Boosting':gbrt}.items()):
    plot_precision_recall_curve(estimator, X_test, y_test, ax=ax[idx])
    ax[idx].set(title=f'{name}')

### `plot_roc_curve`

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20, 5))
for idx, (name, estimator) in enumerate({'Logistic Regression': logreg, 'Random Forest': forest, 'Gradient Boosting':gbrt}.items()):
    plot_roc_curve(estimator, X_test, y_test, ax=ax[idx])
    ax[idx].set(title=f'{name}')

<a id="conclusion"><a/>

# Conclusion












This project resulted in article that can be found [here](link-to-article).